# Imports
The following packages are used:
1. pandas
2. tensorflow
3. cv2
4. matplotlib

In [ ]:
import pandas as pd
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt

# Data
The `license.csv` file consists of the following fields:
* image_name: The path where the image is downloaded; str
* top_x: The % value of the topx bbox point; float
* top_y: The % value of the topy bbox point; float
* bottom_x: The % value of the bottomx bbox point; float
* bottom_y: The % value of the bottomy bbox point; float

In [ ]:
df = pd.read_csv('../input/license/license.csv')
df.head()

# Image Utils
The load img function will read the file, decode it and then resize the file. The user can change the parameters according to their own needs.

In [ ]:
def load_img(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_image(img, channels=3, dtype=tf.float32)
    img = tf.image.resize(img, (224,224))
    if len(img.shape) == 4:
        img = tf.squeeze(img,0)
    img = tf.expand_dims(img, axis=0)
    return img

This snippet is used to concatenate the images and the labels. This is useful as the concatenated tensors will be used to device a `tf.data.Dataset` later.

In [ ]:
# For traininig
imgs = []
labels = []
for index, row in df[:int(0.9*len(df))].iterrows():
    imgs.append(load_img(row['image_name']))
    labels.append(tf.constant([[row['top_x'],row['top_y'],row['bottom_x'],row['bottom_y']]]))
train_images = tf.concat(imgs,axis=0)
train_labels = tf.concat(labels,axis=0)

# For testing
imgs = []
labels = []
for index, row in df[int(0.9*len(df)):].iterrows():
    imgs.append(load_img(row['image_name']))
    labels.append(tf.constant([[row['top_x'],row['top_y'],row['bottom_x'],row['bottom_y']]]))
test_images = tf.concat(imgs,axis=0)
test_labels = tf.concat(labels,axis=0)

This dataset can be used a beginner's step to object detection. For this purpose, I have also included a util function that can help visualize the image and the b-box of the license plates.

In [ ]:
def show_img_bbox(img, label):
    img = img.numpy()
    y_hat = label.numpy()*224
    xt, yt = int(y_hat[0]), int(y_hat[1])
    xb, yb = int(y_hat[2]), int(y_hat[3])
    image = cv2.rectangle(img, (xt, yt), (xb, yb), (0, 0, 255), 3)
    plt.imshow(image)
    plt.show()

# Use the function
show_img_bbox(train_images[13], train_labels[13])

# tf.data.Dataset

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).shuffle(buffer_size=64).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).shuffle(buffer_size=64).batch(32)

In [ ]:
for i,l in train_ds.take(1):
    show_img_bbox(i[0], l[0])
    show_img_bbox(i[2], l[2])